<a href="https://colab.research.google.com/github/tyukei/Aozora-transformer/blob/main/Aozora_transformer_tpu_1000.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/speed1313/jax-llm.git
%cd /content/jax-llm
!pip install -r requirements.lock

fatal: destination path 'jax-llm' already exists and is not an empty directory.
/content/jax-llm
Obtaining file:. (from -r requirements.lock (line 10))
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Using cached jax-0.4.27-py3-none-any.whl.metadata (23 kB)
  Using cached jaxlib-0.4.27-cp312-cp312-manylinux2014_x86_64.whl.metadata (1.8 kB)
Using cached jax-0.4.27-py3-none-any.whl (1.9 MB)
Using cached jaxlib-0.4.27-cp312-cp312-manylinux2014_x86_64.whl (77.4 MB)
  Building editable for jax-llm (pyproject.toml) ... done
  Created wheel for jax-llm: filename=jax_llm-0.1.0-py3-none-any.whl size=4970 sha256=3e69cbd1d9daa967122a2a13990a0378f489c893708123358d6eb6cb707364ff
  Stored in directory: /tmp/pip-ephem-wheel-cache-ado979zr/wheels/cf/7e/54/0a0186b36bb284eb9681ff1ee53ab96675e41de45be35

In [2]:
!python3.12 /content/jax-llm/src/jax_llm/prepare_aozora.py --book_num 1000

Generating train split: 100% 16951/16951 [00:06<00:00, 2809.93 examples/s]
Filter: 100% 16951/16951 [00:03<00:00, 5071.02 examples/s]
1000 books out of 10246 are used


In [4]:
!pip install -U "jax[tpu]==0.4.33" -f https://storage.googleapis.com/jax-releases/libtpu_releases.html
!python3.12  /content/jax-llm/src/jax_llm/train_tokenizer.py --data_name "aozora_1000"

Looking in links: https://storage.googleapis.com/jax-releases/libtpu_releases.html
  Using cached jax-0.4.33-py3-none-any.whl.metadata (22 kB)
  Using cached jaxlib-0.4.33-cp312-cp312-manylinux2014_x86_64.whl.metadata (983 bytes)
Using cached jax-0.4.33-py3-none-any.whl (2.1 MB)
Using cached jaxlib-0.4.33-cp312-cp312-manylinux2014_x86_64.whl (85.1 MB)
  Attempting uninstall: jaxlib
    Found existing installation: jaxlib 0.4.27
    Uninstalling jaxlib-0.4.27:
      Successfully uninstalled jaxlib-0.4.27
  Attempting uninstall: jax
    Found existing installation: jax 0.4.27
    Uninstalling jax-0.4.27:
      Successfully uninstalled jax-0.4.27
[00:00:02] Tokenize words                 ██████████████████ 1238804  /  1238804
[00:00:12] Count pairs                    ██████████████████ 1238804  /  1238804
[00:17:37] Compute merges                 ██████████████████ 44385    /    44385
Total tokens:  8626919


In [6]:
!python3.12 /content/jax-llm/src/jax_llm/train.py \
    --data_name "aozora_1000" \
    --batch_size 32 \
    --n_iterations 2000 \
    --n_freq_eval 100 \
    --dropout_rate 0.1 \
    --learning_rate 0.001 \
    --num_layers 8 \
    --embed_size 384 \
    --head_size 48 \
    --num_heads 8 \
    --block_size 128 \
    --wandb_log False

JAX running on TPU
Tokenizer loaded
Total number of parameters: 48_792_576
  0% 0/2000 [00:00<?, ?it/s]Step: 0	 train loss: 11.38959789276123	 eval loss: 10.542952537536621
  5% 100/2000 [00:27<02:20, 13.51it/s]Step: 100	 train loss: 8.571722030639648	 eval loss: 8.384016036987305
 10% 199/2000 [00:34<02:13, 13.51it/s]Step: 200	 train loss: 8.150879859924316	 eval loss: 8.089750289916992
 15% 299/2000 [00:42<02:05, 13.53it/s]Step: 300	 train loss: 8.06022834777832	 eval loss: 7.63992166519165
 20% 399/2000 [00:49<01:58, 13.53it/s]Step: 400	 train loss: 8.035967826843262	 eval loss: 7.690062522888184
 25% 499/2000 [00:56<01:51, 13.52it/s]Step: 500	 train loss: 7.846991062164307	 eval loss: 7.600899696350098
 30% 599/2000 [01:04<01:43, 13.51it/s]Step: 600	 train loss: 7.7482590675354	 eval loss: 7.524024486541748
 35% 699/2000 [01:11<01:36, 13.52it/s]Step: 700	 train loss: 7.5671067237854	 eval loss: 7.456274509429932
 40% 799/2000 [01:19<01:28, 13.54it/s]Step: 800	 train loss: 7.5011248

In [7]:
!python /content/jax-llm/src/jax_llm/generate.py --prompt "あなたは誰？" --data_name "aozora_1000" --max_new_tokens 200

Output: あなたは 誰 ？ 僕は 、 やっぱり 、 ちゃんと はいって きます 。 「 でも 、 まだ 御 見 だ 。」 と 、 「 いや 、 お いう 」 「 それは これは まだ 、 よく 、 もう その 門 へは 、 しばらく でも 、 「 そうです 。」 「 なに だ 。」 「 うん 、 ああ 、 その 、 あの 小さな 人間 なら 、 この 鍵 の中で 、 その 小さな 巨 人と 顔を 合わせて 、 もう もう 、 まだ その 命 が 、 それは 彼の 探偵 を 要求 だった 。 「 しかし 、 この 人が あります 。 あの 人が 、 よく て さ 。 ただ お 様 、 御 願 を 承 って 、 この 男 、 ただ 、 その 「 それは この 宝 では 、 また 、 それを 、 まるで 、 まだ この 美術 長の 事件 で 、 もう 一人 、 それは 「 それは よく 云 える 。 「 おお 、 あたし と 、 この 「 いや 。 この 、 その 墓 を 、 「 ああ 、 まあ と 、 「 いや 、 あの ものは 、 そして 例の 探偵 の 関係 はない 。 だが 、 この 部屋 にある 。 「 ああ 、 あの 人間が 、 すっかり いる から 、 そして 、 もう 「 ああ 、
